In [0]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [0]:
def conv_block(in_channel, out_channel):
    layer = nn.Sequential(
        nn.BatchNorm2d(in_channel),
        nn.ReLU(True),
        nn.Conv2d(in_channel, out_channel, 3, padding=1, bias = False)
    )
    return layer

In [0]:
class dense_block(nn.Module):
    def __init__(self, in_channel, growth_rate, num_layers):
        super(dense_block, self).__init__()
        block = []
        channel = in_channel
        for i in range(num_layers):
            block.append(conv_block(channel, growth_rate))
            channel += growth_rate
        self.net = nn.Sequential(*block)
    
    def forward(self, x):
        for layer in self.net:
            out = layer(x)
            x = torch.cat((out,x),dim=1)
        return x

In [4]:
test_net = dense_block(3,12,3)
test_x = Variable(torch.zeros(1,3,96,96))
print('input shape:{} x {} x {}'.format(test_x.shape[1],test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print('output shape:{} x {} x {}'.format(test_y.shape[1],test_y.shape[2], test_y.shape[3]))

input shape:3 x 96 x 96
output shape:39 x 96 x 96


#3+12=15      15+12=27      27+12=39

In [0]:
def transition(in_channel, out_channel):
    trans_layer = nn.Sequential(
        nn.BatchNorm2d(in_channel),
        nn.ReLU(True),
        nn.Conv2d(in_channel, out_channel, 1),
        nn.AvgPool2d(2,2)
    )
    return trans_layer

In [6]:
test_net = transition(3,12)
test_x = Variable(torch.zeros(1,3,96,96))
print('input shape:{} x {} x {}'.format(test_x.shape[1],test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print('output shape:{} x {} x {}'.format(test_y.shape[1],test_y.shape[2], test_y.shape[3]))

input shape:3 x 96 x 96
output shape:12 x 48 x 48


In [7]:
print("abc")

abc


In [0]:
class densenet(nn.Module):
    def __init__(self, in_channel, num_classes, growth_rate=32, block_layers=[6,12,24,16]):
        super(densenet, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channel, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(3,2, padding=1)
        )
        
        channels = 64
        block = []
        for i, layers in enumerate(block_layers):
            block.append(dense_block(channels, growth_rate, layers))
            channels += layers * growth_rate
            if i != len(block_layers) - 1:
                block.append(transition(channels, channels // 2))
                channels = channels // 2
        self.block2 = nn.Sequential(*block)
        self.block2.add_module('bn', nn.BatchNorm2d(channels))
        self.block2.add_module('relu', nn.ReLU(True))
        self.block2.add_module('avg_pool', nn.AvgPool2d(3))
        
        self.classifier = nn.Linear(channels, num_classes)
        
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [9]:
test_net = densenet(3, 10)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output:{}'.format(test_y.shape))

output:torch.Size([1, 10])


In [10]:
from Myutils import train

def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = x.transpose((2,0,1))
    x = torch.from_numpy(x)
    return x

train_set = CIFAR10('./data', train=True, transform=data_tf ,download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = densenet(3,10)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified


In [11]:
train(net, train_data, test_data, 20, optimizer, criterion)

/content/Myutils.py:70: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
/content/Myutils.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.370617, Train Acc: 0.506813, Valid Loss: 1.206924, Valid Acc: 0.573774, Time 00:01:43
Epoch 1. Train Loss: 1.003943, Train Acc: 0.644581, Valid Loss: 1.007321, Valid Acc: 0.643097, Time 00:01:53
Epoch 2. Train Loss: 0.837927, Train Acc: 0.707281, Valid Loss: 1.185937, Valid Acc: 0.596816, Time 00:01:53
Epoch 3. Train Loss: 0.711651, Train Acc: 0.751119, Valid Loss: 1.067267, Valid Acc: 0.640724, Time 00:01:53
Epoch 4. Train Loss: 0.616135, Train Acc: 0.784667, Valid Loss: 1.143967, Valid Acc: 0.640625, Time 00:01:53
Epoch 5. Train Loss: 0.528574, Train Acc: 0.817255, Valid Loss: 0.936902, Valid Acc: 0.697884, Time 00:01:53
Epoch 6. Train Loss: 0.450904, Train Acc: 0.844909, Valid Loss: 0.888063, Valid Acc: 0.708762, Time 00:01:53
Epoch 7. Train Loss: 0.385730, Train Acc: 0.866308, Valid Loss: 0.853683, Valid Acc: 0.722903, Time 00:01:53
Epoch 8. Train Loss: 0.332700, Train Acc: 0.885730, Valid Loss: 0.936336, Valid Acc: 0.716179, Time 00:01:53
Epoch 9. Train Loss